In [11]:
#import our libraries
import sys
import time
import psycopg2
import pandas as pd
import numpy as np
import urllib2
import re
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database



In [2]:
def load_boxscore_page():
    print 'here also'

In [3]:
def load_snippet(filename=None, directory=None):

    if filename is not None:
        filename = str(filename)
    else:
        filename = 'sampleboxscorepage_snipet.txt'

    if directory is not None:
        directory = str(directory[0])
    else:
        directory = 'boxscore_pages/'

    target = open(directory+filename, 'r')
    soup = BeautifulSoup(target, 'lxml')

    return soup

In [4]:
def bs_column_names(tag):
    return tag.name == 'th' and tag.has_attr('width') 

In [5]:
def get_column_names(namedata):
    
    column_names = []
    for name in namedata.find_all(bs_column_names):
        match = re.search('.*[A-Z]', str(name))
        if name.get_text() not in column_names: 
            column_names.append(match.group(0).strip())

    return column_names

In [6]:
def bs_column_data(tag):
    return tag.name == 'td' and not tag.has_attr('style')

In [7]:
def get_column_data(datadata):

    column_data = []
    for data in datadata.find_all(bs_column_data):
        match = re.search('.*[0-9]', str(data))
        column_data.append(match.group(0).strip())

    return column_data

In [8]:
def clean_player(mydict):
    mykeys = mydict.keys()
    myvals = mydict.values()

    for ii in np.arange(len(mykeys)):
        match_key = re.search('(.*?)-(.)', mykeys[ii])
        match_value = re.search('(.*?)-(.*)', myvals[ii])
        if match_key is not None:
            mydict[match_key.group(1)] = match_value.group(1)
            newkey = match_key.group(1)
            newkey = newkey.replace(newkey[-1],match_key.group(2))
            mydict[newkey] = match_value.group(2)
    
    return mydict

In [9]:
def build_player(bit1, alldict, team, data_columns):
 
    #print bit1.prettify()
    #team1_columns = get_column_names(bit1)
    #print team1_columns
    column_data = get_column_data(bit1)
    #print column_data
    matches = re.findall(', ([A-Z])', str(bit1))
    #print matches


    for player in bit1.find_all('a'):

        #start a dictionaries
        team1_dict1 = {'team_name':team, 'player':'', 'player_pos':'', 'player_url':''}
        team1_dict2 = {}

        #start filling dictionaries
        team1_dict1['player_url'] = player['href']
        team1_dict1['player'] = player.get_text().strip()
        team1_dict1['player_pos'] = matches[0]
        del matches[0]
        #print team1_dict1
   
        for column in data_columns:
            team1_dict2[column] = column_data[0]
            del column_data[0]
        #print team1_dict2     

        #combine the two dictionaries
        finaldict = team1_dict1.copy()
        finaldict.update(team1_dict2)
        finaldict = clean_player(finaldict)
        alldict.append(finaldict)
    
    return alldict


In [10]:
def build_box_score(soup):

    alldict1 = []    
    box_score = soup.body.div.table
    
    #begin with the first team
    bit1 = box_score.contents[1]
    team = bit1.th.get_text()
    team = team.strip()
    data_columns = get_column_names(bit1)
    alldict1 = build_player(box_score.contents[3], alldict1, team, data_columns)
    alldict1 = build_player(box_score.contents[5], alldict1, team, data_columns)

    #now do the second team
    bit1 = box_score.contents[11]
    team = bit1.th.get_text()
    team = team.strip()
    data_columns = get_column_names(bit1)
    alldict1 = build_player(box_score.contents[13], alldict1, team, data_columns)
    alldict1 = build_player(box_score.contents[15], alldict1, team, data_columns)

    return alldict1

In [11]:
def write_boxscore(outdict, which=None):

    if which is not None:
        which = int(which)
    else: 
        which = 1

    if which == 1:
        df_team1 = pd.DataFrame.from_records(outdict)
        df_team1.to_csv('boxscore_csv/example1.csv')
    
    if which == 2:
        df_team2 = pd.DataFrame.from_records(outdict)
        df_team2.to_csv('boxscore_csv/example2.csv')

In [17]:
def start_games_db(dbname, username):
    print '  Firing up the data base.'
    
    engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
    print '    DB url:', engine.url
    db_exist = database_exists(engine.url)
    if not db_exist:
        create_database(engine.url)
    print '    DB exists? %s' % db_exist
    
    #out_dict = {'dbname':dbname, 'username':username, 
    #            'exists':db_exist, 'engine_url':engine.url, 'engine':engine}
    
    return engine

In [18]:
def main(remake_db=False):
    
    dbname = 'ncaa_mbb_db'
    username = 'smaug'

    boxscore_dir = 'boxscore_pages/'
    boxscore_file = 'ncaa_mbb_boxscore_DDDDDDDDD.txt'
    boxscore_table_name = 'games'

    stats_table_name = 'stats'

    
    if remake_db:
        print '  Now remaking the statistics database.'
        
        #fire up the database engine
        db_engine = start_games_db(dbname, username)
        #print db_engine
        
        #query the boxscore database
    


In [19]:
# boilerplate to execute call to main() function
if __name__ == '__main__':
    main(remake_db=True)

  Now remaking the statistics database.
  Firing up the data base.
    DB url: postgres://smaug@localhost/ncaa_mbb_db
    DB exists? True
Engine(postgres://smaug@localhost/ncaa_mbb_db)
